In [1]:
import numpy as np
import torch 
import sys
sys.path.append("../")

import tqdm
import yaml
torch.random.manual_seed(300)
import matplotlib.pyplot as plt

import utils_BayOTIDE as utils

from model_BayOTIDE import BayTIDE

from model_LDS import LDS_GP_streaming

config_path = "./config_uber.yaml"
with open(config_path, "r") as f:
    config = yaml.safe_load(f)

data_file = config["data_path"] # T=315 - rmse = 0.27 - Mqar23

hyper_dict = utils.make_hyper_dict(config)

data_dict = utils.make_data_dict(hyper_dict,data_file,fold=1)

model = BayTIDE(hyper_dict,data_dict)

model.reset()

INNER_ITER = hyper_dict["INNER_ITER"]
EVALU_T = hyper_dict["EVALU_T"]
for epoch in range(1):
    model.reset()
    for T_id in tqdm.tqdm(range(model.T)):
        model.filter_predict(T_id)
        model.msg_llk_init()

        for inner_it in range(INNER_ITER):

            flag = (inner_it == (INNER_ITER - 1))

            model.msg_approx_W(T_id)
            model.post_update_W(T_id)

            model.msg_approx_U(T_id)
            model.filter_update(T_id,flag)

        model.msg_approx_tau(T_id)
        model.post_update_tau(T_id)

        if T_id % EVALU_T == 0 or T_id == model.T - 1:
            
            # _, loss_dict = model.model_test(T_id)
            # print("T_id = {}, train_rmse = {:.2f}, test_rmse= {:.2f}".format(T_id, loss_dict["train_RMSE"], loss_dict["test_RMSE"]))
            pass

    model.smooth()
    model.post_update_U_after_smooth(0)
    _, loss_dict = model.model_test(T_id)
    print("after smooth, train_rmse = {:.2f}, test_rmse= {:.2f},test_MAE= {:.2f}".format(loss_dict["train_RMSE"], loss_dict["test_RMSE"],loss_dict["test_MAE"]))



use device: cpu


  0%|          | 1/744 [00:00<03:14,  3.82it/s]/home/venus/fang/BayTIDE/notebook/../model_LDS.py:252: UserWarning: An output with one or more elements was resized since it had shape [1, 20, 20], which does not match the required output shape [1, 1, 20, 20]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484806139/work/aten/src/ATen/native/Resize.cpp:17.)
  self.A = torch.matrix_exp(self.F * time_int).double()
 10%|▉         | 74/744 [00:19<02:58,  3.75it/s]


KeyboardInterrupt: 